# Last API with pylast

[Oficial Website](https://www.last.fm/api/)

[PyLast Repo](https://github.com/pylast)

In [ ]:
import pandas as pd 
import numpy as np 
import pylast

import os
import json
import sys
import requests
import time 

sys.path.append('../scripts/') 

from create_last_database import User
from create_last_database import Track
from create_last_database import Artist
from create_last_database import Album
from create_last_database import Tag

## Network with the API through PyLast

In [ ]:
API_KEY = 'e1c7232e1969f803afc123fe4fbd7f18' #input()
API_SECRET = 'dd1404bb182228426ab6b440425dbff1' #input()

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)

Let's get some random users built by `generate_lastfm_users.py`

In [ ]:
def get_random_users(filepath: str, quantity: int = 1000, random_state: int = 200) -> pd.DataFrame:
    
    users = pd.read_csv(filepath)
    chosen_users = users.sample(n = quantity, replace = False, random_state = random_state, axis = 'index')
    chosen_users.index = list(range(0,len(chosen_users)))

    return chosen_users

user_path = "../data/lastfm-api/users_lastfm.csv"

users = get_random_users(user_path)
users.head()

# Creating the database 

It takes a long, long time. Some problems with MalResponse, Network and Connection are expected. For some especial cases, I just rerun the cell. 

We start with the users info. All the following information is saved in a json format.

In [ ]:
users_class = User(network, user_path)

if not os.path.exists('../data/lastfm-api/1k_users_info_lastfm.json'):
    with open('../data/lastfm-api/1k_users_info_lastfm.json', 'w') as f:
        json.dump({}, f)
with open('../data/lastfm-api/1k_users_info_lastfm.json'):
    data = json.load(f)
for user_id, user_name in users.as_matrix():
    if str(user_id) in data:
        continue
    with open('../data/lastfm-api/1k_users_info_lastfm.json', 'r+') as f:
        data = json.load(f)
        user_info = users_class.get_user_info(user_name)
        data[user_id] = user_info
        f.seek(0)
        json.dump(data, f)
        if len(data) % 100 == 0:
            print('{} users - DONE'.format(len(data)))

With the `tracks.csv` file, I will build the tracks dataset. It may take long time!

In [ ]:
track_class = Track(network)

if not os.path.exists('../data/lastfm-api/tracks_lastfm_info.json'):
    with open('../data/lastfm-api/tracks_lastfm_info.json', 'w') as f:
        json.dump({}, f)
with open('../data/lastfm-api/tracks_lastfm_info.json', 'r+') as f:
    data = json.load(f)
for track_id, data_track in track_class.tracks_df.iterrows():
    if str(track_id) in data:
        continue
    with open('../data/lastfm-api/tracks_lastfm_info.json', 'r+') as f:
        t0 = time.time()
        data = json.load(f)
        while True:
            try: 
                track_info = track_class.get_track_info(data_track.track_name, data_track.artist_name)
            except pylast.NetworkError:
                time.sleep(2)
                continue
            break
        data[track_id] = track_info
        f.seek(0)
        json.dump(data, f)
        if len(data) % 100 == 0:
            print('{} tracks - DONE'.format(len(data)))

Building the artist database

In [ ]:
artist_class = Artist(network)

if not os.path.exists('../data/lastfm-api/artists_lastfm_info.json'):
    with open('../data/lastfm-api/artists_lastfm_info.json', 'w') as f:
        json.dump({}, f)
with open('../data/lastfm-api/artists_lastfm_info.json', 'r+') as f:
    data = json.load(f)
for artist_id, data_artist in artist_class.artists_df.iterrows():
    if str(artist_id) in data:
        continue
    with open('../data/lastfm-api/artists_lastfm_info.json', 'r+') as f:
        t0 = time.time()
        data = json.load(f)
        while True:
            try: 
                artist_info = artist_class.get_artist_info(data_artist.artist_name)
            except pylast.NetworkError:
                time.sleep(2)
                continue
            break
        data[artist_id] = artist_info
        f.seek(0)
        json.dump(data, f)
        if len(data) % 100 == 0:
            print('{} artists - DONE'.format(len(data)))

Building the dabase for the tags

In [ ]:
tag_class = Tag(network)

if not os.path.exists('../data/lastfm-api/tags_lastfm_info.json'):
    with open('../data/lastfm-api/tags_lastfm_info.json', 'w') as f:
        json.dump({}, f)
with open('../data/lastfm-api/tags_lastfm_info.json', 'r+') as f:
    data = json.load(f)
for tag_id, data_tag in tag_class.tags_df.iterrows():
    if str(tag_id) in data:
        continue
    with open('../data/lastfm-api/tags_lastfm_info.json', 'r+') as f:
        t0 = time.time()
        data = json.load(f)
        while True:
            try: 
                tag_info = tag_class.get_tag_info(data_tag.tag)
            except pylast.NetworkError:
                time.sleep(2)
                continue
            break
        data[tag_id] = tag_info
        f.seek(0)
        json.dump(data, f)
        if len(data) % 100 == 0:
            print('{} tag - DONE'.format(len(data)))

Converting similar tracks in track info to index. I separate of the original code cause it was lazy!

In [28]:
with open('../data/lastfm-api/tracks_lastfm_info.json', 'r+') as f:
    data = json.load(f)
for key in data.keys():
    if len(data[key]) == 0: 
        continue
    for index, info in enumerate(data[key]['similar']): 
        data[key]['similar'][index] = track_class.get_id_by_name(info[0], info[1])

[21769, 42427, 4392, 13782, 23863, 32058, 11425, 11285, 14656, 31437, 17061, 14618, 15633, 12329, 42428, 29054, 42429, 6113, 4370, 37351]


In [ ]:
with open('../data/lastfm-api/tracks_lastfm_info1.json', 'w') as f: 
    json.dump(data, f)

In [ ]:
with open('../data/lastfm-api/artists_lastfm_info.json', 'r+') as f:
    data = json.load(f)
for key in data.keys():
    if len(data[key]) == 0: 
        continue
    for index, info in enumerate(data[key]['similar']): 
        data[key]['similar'][index] = artist_class.get_id_by_name(info[0])

In [ ]:
with open('../data/lastfm-api/artists_lastfm_info1.json', 'w') as f: 
    json.dump(data, f)